# Deep learning tweeter

In [ ]:
%load_ext tensorboard


## Imports

In [1]:
import sys
import cufflinks
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
import copy
import pickle

warnings.filterwarnings('ignore')
seed = 5
np.random.seed(seed)

sys.path.append('./..')
cufflinks.go_offline()

In [2]:
from Corpus.Corpus import get_corpus, filter_binary_pn, filter_corpus_small
from auxiliar.VectorizerHelper import vectorizer, vectorizerIdf, tokenize, procesar_corpus
from auxiliar import parameters
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import recall_score
from auxiliar.HtmlParser import HtmlParser

In [3]:
import tensorflow as tf
from tensorflow.python.keras.callbacks import TensorBoard

In [4]:
import math
import pickle
import pandas as pd
import Levenshtein as lv
from nltk.stem import SnowballStemmer
from gensim.models import Doc2Vec
import gensim
from nltk.tokenize import word_tokenize
from Corpus.Corpus import get_corpus, filter_binary_pn, filter_corpus_small
from time import time, strftime

In [5]:
import nltk
# nltk.download()

## Config

In [6]:
polarity_dim = 2
# clasificadores=['lstm', '2lstm', '2dcnn', '2dcnn+lstm', 'cnn+lstm', 'bidirectionalLstm']
clasificadores=['lstm']
idf = True
target_names=['Neg', 'Pos']
kfolds = 10
base_dir = '2-clases' if polarity_dim == 2 else ('3-clases' if polarity_dim == 3 else '5-clases')
name = 'deep_learning'

In [7]:
w2vec_file = 'data/w2vec.bin'
stemmer = SnowballStemmer('spanish')

## Get data

Obtenemos los datos para realizar el entrenamiento y lo dividimos en dos subconjuntos: train y test

Dado que nuestro corpus de datos es muy pequeño, 4700 ejemplos para polaridades pos y neg, vamos a dividir el conjunto de datos de forma que utilicemos solo 100 ejemplos para el conjunto de test, asi no penalizaremos al conjunto de entrenamiento.



In [8]:
# cine = HtmlParser(200, "http://www.muchocine.net/criticas_ultimas.php", 1)
data_corpus = get_corpus('general-corpus', 'general-corpus', 1, None)

if polarity_dim == 2:
    data_corpus = filter_binary_pn(data_corpus)
#     cine = filter_binary_pn(cine.get_corpus())
elif polarity_dim == 3:
    data_corpus = filter_corpus_small(data_corpus)
#     cine = filter_corpus_small(cine.get_corpus())
# used_data = cine[:5000]

#Intentando obtener datos del archivo csv...
./../Corpus/../data/general-corpus.csv
#Datos recuperados!


In [9]:
data_corpus.groupby('polarity').agg({'index': 'count'}).iplot(kind='bar')

## Preprocess

Usamos la función axiliar prepro (VectorizerHelper.procesar_corpus) con los siguientes parámetros:

text, tagger, process_text, stop_words, negation, repeated_letters

In [1]:
def apply_prepro(data):
    return procesar_corpus(data, True, True, False, True)
data_corpus.content = data_corpus.content.apply(apply_prepro)

NameError: name 'data_corpus' is not defined

### Model initialization

In [ ]:
model = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format(w2vec_file, binary=True)

### Tokenize texts

Utilizamos la funciona auxiliar tokenize que utiliza como parametros el conjunto de datos y un flag para activar el stemming.

In [ ]:
def apply_tokenization(data):
    return tokenize(data, False)
tokens = data_corpus.content.apply(apply_tokenization)

In [ ]:
token_df = pd.DataFrame([x for x in tokens]).transpose()
token_df.columns = pd.MultiIndex.from_arrays([data_corpus.polarity, token_df.columns])

In [ ]:
print('palabras totales', token_df.count().sum())
print('media de palabras por texto', token_df.count().mean())

In [ ]:
token_df.count().iplot(kind='histogram')

A pesar de que la mayoría de documentos tienen un máximo de 10 palabras, vamos a utilizar las 28 de máximo debido a la poca cantidad de datos de entrenamiento de la que disponemos

In [ ]:
pd.DataFrame([token_df[x].count().sum() for x in token_df.columns.levels[0]]).iplot(kind='bar')

La cantidad de palabras por polaridad es bastante similar, aunque la diferencia puede llegar a suponer un problema en el resultado final según la distribución de los conjuntos de entrenamiento y de test

### w2vec process

Obtenemos las palabras que no se encuentran en el diccionario, probablemente por estar mal escritas como enorahuena en lugar de enhorabuena, en este caso los stems serían enorahuen y enhorabuen respectivamente, como vemos hay una diferencia de dos letras

Basandonos en esto vamos a establecer una diferencia máxima de 3 letras para considerar la palabra sustituible por la palabra correcta

In [ ]:
# stem_vocab = np.array([stemmer.stem(x) for x in model.vocab])
stem_vocab = np.array([x for x in model.vocab])
stem_vocab_dict = dict.fromkeys(stem_vocab, 1)

In [ ]:
asci_codes = [np.array([ord(x) for x in y]) for y in stem_vocab]

La palabra más larga del español según la RAE es electroencefalografista

In [ ]:
len("electroencefalografista")

In [ ]:
padded_asci_codes = tf.keras.preprocessing.sequence.pad_sequences(asci_codes, maxlen=23, padding='post')

In [ ]:
from datetime import datetime
from tensorflow.python.keras.callbacks import TensorBoard
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(word_index), 300, input_length=12, weights=[weight_matrix], name='Embedding', trainable=False),
    tf.keras.layers.LSTM(64, activation='relu', name='lstm'),
    tf.keras.layers.Dense(1, activation='sigmoid' ,name='dense')
])
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir, embeddings_freq=1, write_images=True)

#### No encontradas en vocabulario

In [ ]:
not_in_vocab = pd.concat([token_df[d][token_df[d].apply(lambda x: x not in stem_vocab_dict)] for d in token_df.columns], axis=1)

In [ ]:
print('total de palabras no encontradas en el vocabulario', not_in_vocab.count().sum())
print('media de palabras no encontradas en el conjunto', not_in_vocab.count().mean())

In [ ]:
pd.DataFrame([not_in_vocab[x].count().sum() for x in not_in_vocab.columns.levels[0]]).iplot(kind='bar')

In [ ]:
not_in_vocab.columns = not_in_vocab.columns.droplevel()

In [ ]:
not_in_vocab_words = pd.DataFrame(
    [x for sublist in [not_in_vocab[y].dropna().values for y in not_in_vocab.columns] for x in sublist]
).drop_duplicates()

#### Encontradas en vocabulario

In [ ]:
in_vocab = pd.concat([token_df[d][token_df[d].apply(lambda x: x in stem_vocab_dict)] for d in token_df.columns], axis=1)

In [ ]:
in_vocab.columns = in_vocab.columns.droplevel()

In [ ]:
in_vocab_words = pd.DataFrame(
    [x for sublist in [in_vocab[y].dropna().values for y in in_vocab.columns] for x in sublist]
).drop_duplicates()

In [ ]:
in_vocab.head()

#### Replace process

In [ ]:
def replace_words(w):
    found_positions = np.where(stem_vocab == w)[0]
    return model[vocab_keys[found_positions[0]]]

In [ ]:
def custom_levenshtein(word, dictionary):
    ascii_matrix = np.repeat([word], [dictionary.shape[0]], axis=0)
    difference = ascii_matrix - dictionary
    difference = np.where(difference != 0, 1, difference)
    difference_sum = np.sum(difference, axis=1)
    minval = np.min(difference_sum)
    minidx = np.argmin(difference_sum)
    if minval < 2 and minval > 0:
        return minidx
    else:
        return None

In [ ]:
def find_coincidences(word, dictionary):
    min_len = max(len(word) - 1, 1)
    max_len = len(word) + 1
    dist = np.array([lv.distance(x, word) - x.startswith(word) if len(x) > min_len and len(x) < max_len else 999 for x in dictionary])
    minval = np.min(dist)
    if minval < 2 and minval > 0:
        minidx = np.argmin(dist)
#         root_logger.info('current word %s - found_distance %s - idx to replace %s word', word, minval, minidx)
        return dictionary[minidx], np.delete(dictionary, minidx)
    else:
        return None, dictionary
    
    
    

In [ ]:
import gc
gc.collect()

In [ ]:
dictionary = not_in_vocab_words.values.reshape(1,-1)[0]
ascii_dictionary = np.array([np.array([ord(x) for x in y]) for y in dictionary])
padded_dictionary = tf.keras.preprocessing.sequence.pad_sequences(ascii_dictionary, maxlen=23, padding="post")

found = dict()
with tqdm(total=len(padded_asci_codes)) as pbar:
    for i, val in enumerate(padded_asci_codes):
        pbar.update(1)
        coincidence = custom_levenshtein(val, padded_dictionary)
        if coincidence:
            found[dictionary[coincidence]] = model[stem_vocab[i]]
        if len(found) == dictionary.shape[0]:
            break

In [ ]:
gc.collect()

In [ ]:
dictionary = in_vocab_words.values.reshape(1,-1)[0]
w2vec_found = dict()
with tqdm(total=len(dictionary)) as pbar:
    for word in dictionary:
        pbar.update(1)
        w2vec_found[word] = model[word]

In [ ]:
in_vocab_replaced = in_vocab.applymap(lambda x: w2vec_found[x] if x in w2vec_found else math.nan)

In [ ]:
not_in_vocab_replaced = not_in_vocab.applymap(lambda x: found[x] if x in found else math.nan)

In [ ]:
in_vocab_replaced.update(not_in_vocab_replaced)

#### Replace nan positions

Todos los textos deben tener la misma longitud, por lo que aquellos textos que no tienen la longitud indicanda deben rellenarse con arrays de 0 de una dimension establecida (la misma para todas las palabras)

In [ ]:
nan_pos = pd.DataFrame([in_vocab_replaced[c].isna() for c in in_vocab_replaced.columns]).transpose()

In [ ]:
features = 300
zeros = np.zeros((features))
in_vocab_replaced.update(nan_pos.applymap(lambda x: zeros if x else math.nan))

In [ ]:
in_vocab_replaced.to_pickle('data/pkls/tweeter_wemb.pkl')

## Split data

In [9]:
in_vocab_replaced = pd.read_pickle('data/pkls/tweeter_wemb.pkl')

In [10]:
in_vocab_replaced.head()

,0,1,2,3,4,5,6,7,8,9,...,4690,4691,4692,4693,4694,4695,4696,4697,4698,4699
0,"[0.123253495, 0.047755074, 0.18744704, -0.0576...","[-0.03825481, 0.4745884, 0.06159374, -0.211678...","[0.08140966, -0.2937571, 0.09323869, -0.070561...","[-0.08855907, -0.04540643, -0.03799705, 0.1179...","[-0.24166, -0.31998757, 0.05182405, -0.0511, 0...","[-0.30192834, -0.091402225, -0.07635854, -0.32...","[0.16187154, -0.15262279, 0.15911105, 0.136880...","[-0.03385875, -0.05679143, 0.15936868, 0.03850...","[0.08511154, -0.5224435, -0.1114207, -0.029714...","[-0.24166, -0.31998757, 0.05182405, -0.0511, 0...",...,"[0.08881656, -0.08638146, 0.19055837, 0.011414...","[0.24844696, -0.045677852, 0.023781389, -0.055...","[0.028837433, -0.1533759, -0.15925558, -0.0515...","[0.084870994, 0.018372163, -0.19153509, -0.140...","[0.19372217, -0.00981669, 0.0936164, 0.0327220...","[0.2661146, 0.10789581, 0.24465632, 0.09246798...","[0.0937914, -0.06750509, 0.11355269, -0.071939...","[0.05255723, -0.173229, -0.043076243, -0.07914...","[0.08545774, -0.18351299, 0.040896367, -0.2875...","[0.19491133, 0.13588089, 0.26361302, 0.0549132..."
1,"[-0.27860123, -0.0073691155, 0.07620924, -0.18...","[-0.13225149, 0.007982017, -0.15443377, -0.041...","[-0.22758521, 0.119482145, 0.07687994, -0.1562...","[0.19492386, 0.34440613, 0.05423296, 0.1994891...","[0.11887759, -0.062084418, 0.24743606, 0.08697...","[-0.03682754, 0.124864206, 0.08532753, 0.11072...","[0.29450724, -0.08953724, 0.22900815, -0.13844...","[-0.01992176, -0.38204813, 0.08824053, 0.02404...","[0.032929733, 0.071419924, -0.063104734, -0.08...","[0.16452287, 0.04510333, 0.17681116, -0.175676...",...,"[0.15753947, -0.11052575, 0.026150983, -0.0067...","[0.18700868, -0.04345352, -0.21399334, -0.0299...","[-0.115100406, 0.04057121, -0.051373735, -0.14...","[0.008335834, -0.13954785, 0.07759602, -0.0191...","[0.057889074, -0.0860811, 0.005500754, 0.19180...","[-0.09766652, -0.055921096, -0.18628502, -0.33...","[-0.14751814, 0.014919235, -0.06650046, -0.193...","[0.20185633, 0.16108106, 0.17906275, 0.0547961...","[-0.015799766, 0.09396541, 0.08841807, -0.0635...","[0.2661146, 0.10789581, 0.24465632, 0.09246798..."
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.1472689, 0.035210133, -0.0905985, 0.235152...","[-0.19423386, -0.1605221, -0.122799665, -0.226...","[0.010192282, 0.011454537, -0.024294477, -0.14...","[-0.016709665, 0.04634203, 0.12317722, -0.1102...","[0.015565158, 0.013814226, 0.2070413, -0.10364...","[0.23736276, 0.31505284, -0.36881423, 0.341764...","[-0.2906534, 0.38485017, -0.32198793, -0.46555...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[-0.15636458, -0.14417209, 0.15087254, -0.0876...","[-0.18826456, -0.26298478, -0.30475214, -0.043...","[-0.32796225, 0.12644982, 0.1373805, -0.328892...","[-0.021904068, -0.28292415, 0.036686286, -0.12...","[0.31308195, 0.28671673, 0.061079856, -0.18517...","[0.060204167, -0.17087598, 0.2108287, -0.02707...","[-0.09252765, -0.06127216, -0.21862276, -0.010...","[0.10744205, 0.0075490335, -0.28650734, 0.2955...","[0.025537454, -0.1880749, 0.1568079, -0.178805...","[-0.06769281, -0.30116892, 0.08937262, 0.08235..."
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.038064547, -0.045823276, -0.07485215, -0.07...","[0.25501376, -0.011608973, -0.27930972, 0.3525...","[-0.18029697, 0.08496069, 0.01006801, -9.25252...","[-0.25586495, 0.28853804, 0.059111107, -0.0686...","[0.055961747, 0.03338554, 0.17965522, -0.18108...","[-0.0312227, -0.05622646, 0.2197303, -0.072537...","[-0.17901269, -0.015474343, 0.004863352, 0.029...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[-0.2848985, -0.20139082, -0.099536225, 0.1538...","[0.08226747, 0.031416256, 0.0009123865, -0.560...","[0.40826887, -0.085287675, 0.13206185, -0.1375...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.16144432, -0.056896593, -0.029683227, -0.1...","

Dado que solo tenemos 4700 elementos, lo que resulta ser un corpus bastante pequeño, vamos a quedarnos con solo 100 elementos como conjunto de test y utilizaremos el resto como conjunto de entrenamiento

In [11]:
split = in_vocab_replaced.shape[1] * 0.7

In [12]:
train_corpus = in_vocab_replaced.loc[:, :split - 1]
test_corpus = in_vocab_replaced.loc[:, split:]

In [13]:
print("corpus shape ", in_vocab_replaced.shape)
print("train_corpus shape ", train_corpus.shape)
print("test_corpus shape ", test_corpus.shape)

corpus shape  (28, 4700)
train_corpus shape  (28, 3290)
test_corpus shape  (28, 1410)


In [14]:
features=train_corpus[0][0].shape[0]
timesteps=train_corpus.shape[0]
elements=train_corpus.shape[1]
print("features ", features)
print("timesteps ", timesteps)
print("elements ", elements)

features  300
timesteps  28
elements  3290


In [15]:
wemb_x = np.array([np.concatenate(train_corpus[x].values) for x in train_corpus.columns])

In [16]:
wemb_test = np.array([np.concatenate(test_corpus[x].values) for x in test_corpus.columns])

In [17]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()

In [18]:
tokenizer.fit_on_texts(data_corpus.content)

In [19]:
sequences = tokenizer.texts_to_sequences(data_corpus.content)

In [20]:
sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=28, padding='post')

In [21]:
sequences.shape

(4700, 28)

In [22]:
pd.DataFrame(tokenizer.word_index.items(), columns=["word", "idx"]).to_pickle('data/pkls/word_index.pkl')

## Get final train data

In [23]:
wemb_x.shape

(3290, 8400)

In [24]:
x_train = wemb_x.reshape(elements, timesteps, features)

In [25]:
x_train.shape

(3290, 28, 300)

In [26]:
x_test = wemb_test.reshape(test_corpus.shape[1], timesteps, features)

In [27]:
x_test.shape

(1410, 28, 300)

In [28]:
from sklearn.preprocessing import LabelEncoder

In [29]:
def get_Y(corpus, polarity_dim=polarity_dim):
    if polarity_dim == 2:
        encoder = LabelEncoder()
        encoder.fit(corpus.polarity.values)
        Y = encoder.transform(corpus.polarity.values)
        return Y
    else:
        return to_categorical([x for x in train.polarity.values])

In [30]:
Y = get_Y(data_corpus, polarity_dim)

In [31]:
Y.shape

(4700,)

In [32]:
y_train = Y[:3290]
y_test = Y[3290:]

In [33]:
print('y_train shape ', y_train.shape)
print('y_test shape ', y_test.shape)

y_train shape  (3290,)
y_test shape  (1410,)


In [34]:
pd.DataFrame(y_train).iplot(kind='histogram')

In [35]:
pd.DataFrame(y_test).iplot(kind='histogram')

### Definición de funciones

In [36]:
colors=['red', 'blue','red', 'blue','red', 'blue','red', 'blue','red', 'blue','red', 'blue','red', 'blue','red', 'blue','red', 'blue','red', 'blue']

In [1]:
kf = KFold(n_splits=kfolds, shuffle=True, random_state=None) # realización de k-folds
folds = kf.split(x_train)
folds = pd.read_pickle('data/pkls/folds.pkl')

NameError: name 'KFold' is not defined

In [38]:
folds = folds.values

In [39]:
folds

array([[array([   1,    2,    3, ..., 3286, 3287, 3288]),
        array([   0,    6,    9,   20,   47,   68,   79,   97,  111,  115,  142,
        146,  163,  165,  169,  170,  175,  178,  183,  220,  227,  230,
        231,  232,  278,  280,  284,  286,  295,  297,  321,  325,  329,
        338,  347,  356,  366,  376,  378,  385,  394,  412,  421,  426,
        432,  443,  463,  464,  466,  467,  472,  480,  514,  517,  521,
        546,  559,  563,  575,  579,  580,  594,  599,  602,  603,  621,
        629,  641,  646,  651,  653,  675,  680,  681,  687,  695,  703,
        720,  722,  733,  752,  755,  773,  791,  796,  797,  819,  843,
        847,  863,  890,  896,  907,  908,  944,  945,  953,  983,  994,
       1000, 1001, 1006, 1008, 1015, 1021, 1059, 1062, 1064, 1066, 1083,
       1090, 1103, 1104, 1108, 1110, 1128, 1132, 1155, 1156, 1163, 1180,
       1186, 1188, 1191, 1200, 1201, 1236, 1266, 1267, 1276, 1279, 1293,
       1300, 1302, 1312, 1315, 1325, 1333, 1334, 1338, 134

In [40]:
def convert_to_df(hist):
    cols = pd.MultiIndex.from_product([["step1", "step2", "step3", "step4", "step5", "step6", "step7", "step8", "step9", "step10"], hist[0].history.keys()])
    hist_df = pd.concat([pd.DataFrame(x.history) for x in  hist], axis=1)
    hist_df.columns = cols
    hist_df.head()
    return hist_df

In [41]:
def compute_metrics(predictions, real):
    metrics = dict()
    bin_preds = [1 if x[0] >= 0.5 else 0 for x in predictions]
    metrics['mse'] = mean_squared_error(bin_preds, real)
    metrics['recall'] = recall_score(bin_preds, real)
    metrics['f1'] = f1_score(bin_preds, real)
    metrics['acc'] = accuracy_score(bin_preds, real)
    return metrics
    

In [42]:
def kfold_train(model_func, model_name, **params):
    evaluations = list()
    hists = list()
    i = 0
    for train_index, val_index in folds:
        i += 1
        model = model_func()
        train_x = x_train[train_index]
        train_y = y_train[train_index]
        val_x = x_train[val_index]
        val_y = y_train[val_index]
        
        logdir="logs/DL/"+ model_name+"/kfold" + str(i)
        print(logdir)
#         tensorboard_callback = TensorBoard(log_dir=logdir)

#         hist = model.fit(train_x, train_y, validation_data=(val_x, val_y), callbacks=[tensorboard_callback], **params)
        hist = model.fit(train_x, train_y, validation_data=(val_x, val_y), **params)
        
        hists.append(hist)
        evaluations.append(compute_metrics(model.predict(x_test), y_test))
    hist_df = convert_to_df(hists)
    hist_df.to_pickle('results/'+name+'/tweeter/' + base_dir + '/'+model_name+'_lstm.pkl')
    evas_df = pd.DataFrame.from_dict(evaluations)
    evas_df.to_pickle('results/'+name+'/tweeter/' + base_dir + '/'+model_name+'_lstm_evas.pkl')    
    return hist_df, evas_df

In [47]:
def compute_full_model(model_func, model_name, **params):
    model = model_func()
    model.fit(x_train, y_train, **params)
    model.save('results/'+name+'/tweeter/' + base_dir + '/' + model_name + '.h5' )
    preds = model.predict(x_test)
    pd.DataFrame(preds).to_pickle('results/'+name+'/tweeter/' + base_dir + '/' + model_name + '_preds.pkl')
    return preds

In [44]:
def plot_loss(hist_df):
    hist_df.loc[:, pd.IndexSlice[:, ['loss', 'val_loss']]].iplot(colors=colors)
    print('mean\n', hist_df.stack(level=0).mean())
    print('std\n', hist_df.stack(level=0).std())

In [45]:
def plot_val(evas_df):
    evas_df.iplot()
    print('media\n', evas_df.mean())
    print('std\n', evas_df.std())

In [3]:
def process_results(model_name):
    hist = pd.read_pickle('results/'+name+'/tweeter/' + base_dir + '/'+ model_name + '_lstm.pkl')
    evas = pd.read_pickle('results/'+name+'/tweeter/' + base_dir + '/'+ model_name + '_lstm_evas.pkl')
    plot_loss(hist)
    plot_val(evas)

### Definición de modelos

#### Linea base

In [48]:
def create_lstm_val():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, kernel_initializer='normal', activation='relu', input_shape=(28, 300), name='lstm'),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='normal', name='dense')
    ])
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
    return model

In [49]:
val_hist, val_evas = kfold_train(create_lstm_val, 'lstm_val', batch_size=256, epochs=30, shuffle=False, verbose=2)

W0628 21:06:29.992429 139685619763008 deprecation.py:506] From /home/suampa/.local/lib/python3.7/site-packages/tensorflow_core/python/keras/initializers.py:143: calling RandomNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0628 21:06:30.272864 139685619763008 deprecation.py:323] From /home/suampa/.local/lib/python3.7/site-packages/tensorflow_core/python/ops/nn_impl.py:182: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


logs/DL/lstm_val/kfold1
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 2s - loss: 0.6925 - acc: 0.5708 - val_loss: 0.6903 - val_acc: 0.6049
Epoch 2/30
2961/2961 - 1s - loss: 0.6878 - acc: 0.5731 - val_loss: 0.6569 - val_acc: 0.6049
Epoch 3/30
2961/2961 - 1s - loss: 0.7068 - acc: 0.5795 - val_loss: 0.6819 - val_acc: 0.6383
Epoch 4/30
2961/2961 - 1s - loss: 0.8461 - acc: 0.6863 - val_loss: 1.0139 - val_acc: 0.6960
Epoch 5/30
2961/2961 - 1s - loss: 0.7413 - acc: 0.6217 - val_loss: 0.6717 - val_acc: 0.6049
Epoch 6/30
2961/2961 - 1s - loss: 0.6719 - acc: 0.5731 - val_loss: 0.6308 - val_acc: 0.6049
Epoch 7/30
2961/2961 - 1s - loss: 0.6472 - acc: 0.5731 - val_loss: 0.6359 - val_acc: 0.6049
Epoch 8/30
2961/2961 - 1s - loss: 0.6232 - acc: 0.5731 - val_loss: 0.5926 - val_acc: 0.6049
Epoch 9/30
2961/2961 - 1s - loss: 0.5862 - acc: 0.5731 - val_loss: 0.5660 - val_acc: 0.6049
Epoch 10/30
2961/2961 - 1s - loss: 0.5527 - acc: 0.5731 - val_loss: 0.5607 - val_acc: 0.6049
Epoch 11

2961/2961 - 3s - loss: 0.3747 - acc: 0.8423 - val_loss: 0.3826 - val_acc: 0.8450
Epoch 28/30
2961/2961 - 3s - loss: 0.3208 - acc: 0.8595 - val_loss: 0.4070 - val_acc: 0.8359
Epoch 29/30
2961/2961 - 3s - loss: 0.2674 - acc: 0.8811 - val_loss: 0.4048 - val_acc: 0.8207
Epoch 30/30
2961/2961 - 3s - loss: 0.3197 - acc: 0.8609 - val_loss: 0.3819 - val_acc: 0.8328
logs/DL/lstm_val/kfold4
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 1s - loss: 0.6924 - acc: 0.5647 - val_loss: 0.6909 - val_acc: 0.5897
Epoch 2/30
2961/2961 - 0s - loss: 0.6898 - acc: 0.6305 - val_loss: 0.6963 - val_acc: 0.6474
Epoch 3/30
2961/2961 - 0s - loss: 0.6822 - acc: 0.6197 - val_loss: 0.6808 - val_acc: 0.5897
Epoch 4/30
2961/2961 - 0s - loss: 0.6859 - acc: 0.5762 - val_loss: 0.6629 - val_acc: 0.5866
Epoch 5/30
2961/2961 - 0s - loss: 0.6684 - acc: 0.5802 - val_loss: 0.6672 - val_acc: 0.5927
Epoch 6/30
2961/2961 - 0s - loss: 0.6579 - acc: 0.5832 - val_loss: 0.6278 - val_acc: 0.5957
Epoch 7/30
2961/2

Epoch 24/30
2961/2961 - 3s - loss: 0.1819 - acc: 0.9230 - val_loss: 1.1259 - val_acc: 0.8267
Epoch 25/30
2961/2961 - 3s - loss: 0.1651 - acc: 0.9318 - val_loss: 0.9247 - val_acc: 0.8207
Epoch 26/30
2961/2961 - 3s - loss: 0.1811 - acc: 0.9200 - val_loss: 0.9527 - val_acc: 0.8237
Epoch 27/30
2961/2961 - 3s - loss: 0.1777 - acc: 0.9216 - val_loss: 0.7343 - val_acc: 0.8024
Epoch 28/30
2961/2961 - 3s - loss: 0.2603 - acc: 0.8798 - val_loss: 0.4478 - val_acc: 0.8237
Epoch 29/30
2961/2961 - 3s - loss: 0.1936 - acc: 0.9213 - val_loss: 0.7681 - val_acc: 0.8267
Epoch 30/30
2961/2961 - 3s - loss: 0.1458 - acc: 0.9375 - val_loss: 0.7810 - val_acc: 0.8024
logs/DL/lstm_val/kfold7
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 8s - loss: 0.6925 - acc: 0.5596 - val_loss: 0.6907 - val_acc: 0.5836
Epoch 2/30
2961/2961 - 3s - loss: 0.6881 - acc: 0.5799 - val_loss: 0.6757 - val_acc: 0.6748
Epoch 3/30
2961/2961 - 3s - loss: 0.6940 - acc: 0.6694 - val_loss: 0.6526 - val_acc: 0.5836
Ep

2961/2961 - 0s - loss: 0.3272 - acc: 0.8538 - val_loss: 0.4541 - val_acc: 0.8176
Epoch 21/30
2961/2961 - 0s - loss: 0.3387 - acc: 0.8555 - val_loss: 0.4139 - val_acc: 0.8146
Epoch 22/30
2961/2961 - 0s - loss: 0.3167 - acc: 0.8629 - val_loss: 0.4454 - val_acc: 0.8328
Epoch 23/30
2961/2961 - 0s - loss: 0.2705 - acc: 0.8818 - val_loss: 0.4455 - val_acc: 0.8267
Epoch 24/30
2961/2961 - 0s - loss: 0.2570 - acc: 0.8906 - val_loss: 0.5224 - val_acc: 0.8237
Epoch 25/30
2961/2961 - 0s - loss: 0.2417 - acc: 0.8943 - val_loss: 0.5035 - val_acc: 0.8207
Epoch 26/30
2961/2961 - 0s - loss: 0.2320 - acc: 0.9004 - val_loss: 0.5540 - val_acc: 0.8176
Epoch 27/30
2961/2961 - 0s - loss: 0.2316 - acc: 0.9048 - val_loss: 0.6166 - val_acc: 0.8207
Epoch 28/30
2961/2961 - 0s - loss: 0.2103 - acc: 0.9095 - val_loss: 0.5860 - val_acc: 0.8176
Epoch 29/30
2961/2961 - 0s - loss: 0.1944 - acc: 0.9159 - val_loss: 0.8305 - val_acc: 0.8328
Epoch 30/30
2961/2961 - 0s - loss: 0.1863 - acc: 0.9193 - val_loss: 0.7358 - val_a

In [2]:
process_results('lstm_val')

NameError: name 'process_results' is not defined

In [51]:
val_pred = compute_full_model(create_lstm_val, 'lstm_val', batch_size=256, epochs=30, shuffle=False, verbose=1)
compute_metrics(val_pred, y_test)

Train on 3290 samples
Epoch 1/30
3290/3290 [==============================] - 6s 2ms/sample - loss: 0.6922 - acc: 0.5720
Epoch 2/30
3290/3290 [==============================] - 3s 974us/sample - loss: 0.6881 - acc: 0.6714
Epoch 3/30
3290/3290 [==============================] - 3s 946us/sample - loss: 0.6811 - acc: 0.5842
Epoch 4/30
3290/3290 [==============================] - 3s 958us/sample - loss: 0.6580 - acc: 0.5766
Epoch 5/30
3290/3290 [==============================] - 3s 962us/sample - loss: 0.5926 - acc: 0.6015
Epoch 6/30
3290/3290 [==============================] - 3s 936us/sample - loss: 0.5363 - acc: 0.6845
Epoch 7/30
3290/3290 [==============================] - 3s 956us/sample - loss: 0.5093 - acc: 0.7845
Epoch 8/30
3290/3290 [==============================] - 3s 950us/sample - loss: 0.5058 - acc: 0.7736
Epoch 9/30
3290/3290 [==============================] - 3s 949us/sample - loss: 0.4752 - acc: 0.7951
Epoch 10/30
3290/3290 [==============================] - 3s 949us/sampl

{'mse': 0.19787234042553192,
 'recall': 0.9484193011647255,
 'f1': 0.8033826638477801,
 'acc': 0.8021276595744681}

La reducción de la variable loss en el conjunto de entrenamiento produce un efecto de sobreentrenamiento que provoca un mayor error en el conjunto de validación

Algunas de las opciones para reducir este efecto son:
* Reducir la complejidad de la red neuronal
* Aplicar alguna clase de regularización al modelo
* Buscar una topología que se adapte mejor al problema
* Obtener más datos

#### Reducción complejidad modelo

In [53]:
def create_simpler_model():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(10, kernel_initializer='normal', activation='relu', input_shape=(28, 300), name='lstm'),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='normal', name='dense')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
simpler_hist, simpler_evas = kfold_train(create_simpler_model, 'lstm_simple', batch_size=256, epochs=30, shuffle=False, verbose=2)

In [78]:
process_results('lstm_simple')

mean
 acc         0.739269
loss        0.488603
val_acc     0.727224
val_loss    0.536580
dtype: float64
std
 acc         0.111174
loss        0.140881
val_acc     0.096400
val_loss    0.113342
dtype: float64


media
 acc       0.798227
f1        0.821239
mse       0.201773
recall    0.845267
dtype: float64
std
 acc       0.027182
f1        0.027168
mse       0.027182
recall    0.038591
dtype: float64


In [54]:
simpler_pred = compute_full_model(create_simpler_model, 'lstm_simple', batch_size=256, epochs=30, shuffle=False, verbose=1)
compute_metrics(simpler_pred, y_test)

Train on 3290 samples
Epoch 1/30
3290/3290 [==============================] - 1s 350us/sample - loss: 0.6927 - acc: 0.5726
Epoch 2/30
3290/3290 [==============================] - 1s 184us/sample - loss: 0.6917 - acc: 0.5763
Epoch 3/30
3290/3290 [==============================] - 1s 189us/sample - loss: 0.6905 - acc: 0.5763
Epoch 4/30
3290/3290 [==============================] - 1s 163us/sample - loss: 0.6892 - acc: 0.5763
Epoch 5/30
3290/3290 [==============================] - 1s 154us/sample - loss: 0.6872 - acc: 0.5763
Epoch 6/30
3290/3290 [==============================] - 1s 154us/sample - loss: 0.6679 - acc: 0.5763
Epoch 7/30
3290/3290 [==============================] - 0s 143us/sample - loss: 0.6373 - acc: 0.5763
Epoch 8/30
3290/3290 [==============================] - 0s 137us/sample - loss: 0.6052 - acc: 0.5763
Epoch 9/30
3290/3290 [==============================] - 0s 136us/sample - loss: 0.5796 - acc: 0.5763
Epoch 10/30
3290/3290 [==============================] - 0s 142us/sam

{'mse': 0.18581560283687942,
 'recall': 0.8746630727762803,
 'f1': 0.8320512820512821,
 'acc': 0.8141843971631205}

La reducción de dimensionalidad de la red neuronal permite reducir la velocidad de aprendizaje sobre el conjunto de entrenamiento reduciendo así la perdida en el conjunto de validación.

Si lo compramos con el resultado anterior vemos que en general se reducen los picos negativos, en un 0.30 como minimo, sin embargo la convergencia sigue estando sobre 0.4 en los mejores casos

#### Regularización por dropout

In [55]:
def create_drop_model():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(10, kernel_initializer='normal', activation='relu', input_shape=(28, 300), name='lstm'),
        tf.keras.layers.Dropout(0.7),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='normal', name='dense')
    ])
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
drop_hist, drop_evas = kfold_train(create_drop_model, 'dropout_lstm', batch_size=256, epochs=30, shuffle=False, verbose=1)

In [79]:
process_results('dropout_lstm')

mean
 acc         0.652630
loss        0.598253
val_acc     0.692827
val_loss    0.588355
dtype: float64
std
 acc         0.072949
loss        0.089739
val_acc     0.102583
val_loss    0.104724
dtype: float64


media
 acc       0.785887
f1        0.824628
mse       0.214113
recall    0.810059
dtype: float64
std
 acc       0.064060
f1        0.029121
mse       0.064060
recall    0.086830
dtype: float64


In [56]:
drop_pred = compute_full_model(create_drop_model, 'dropout_lstm', batch_size=256, epochs=30, shuffle=False, verbose=1)
compute_metrics(drop_pred, y_test)

W0628 21:55:31.228050 139685619763008 nn_ops.py:4230] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Train on 3290 samples
Epoch 1/30
3290/3290 [==============================] - 1s 405us/sample - loss: 0.6928 - acc: 0.5726
Epoch 2/30
3290/3290 [==============================] - 1s 209us/sample - loss: 0.6918 - acc: 0.5760
Epoch 3/30
3290/3290 [==============================] - 1s 214us/sample - loss: 0.6886 - acc: 0.5933
Epoch 4/30
3290/3290 [==============================] - 1s 184us/sample - loss: 0.6905 - acc: 0.5991
Epoch 5/30
3290/3290 [==============================] - 1s 190us/sample - loss: 0.6883 - acc: 0.5772
Epoch 6/30
3290/3290 [==============================] - 1s 180us/sample - loss: 0.6870 - acc: 0.5766
Epoch 7/30
3290/3290 [==============================] - 1s 259us/sample - loss: 0.6853 - acc: 0.5784
Epoch 8/30
3290/3290 [==============================] - 1s 200us/sample - loss: 0.6788 - acc: 0.6067
Epoch 9/30
3290/3290 [==============================] - 1s 244us/sample - loss: 0.6716 - acc: 0.6210
Epoch 10/30
3290/3290 [==============================] - 1s 298us/sam

{'mse': 0.1723404255319149,
 'recall': 0.8797886393659181,
 'f1': 0.8457142857142856,
 'acc': 0.8276595744680851}

Un valor tan alto de dropout (0.7) provoca peores resultados

In [57]:
def create_drop_model_2():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(10, kernel_initializer='normal', activation='relu', input_shape=(28, 300), name='lstm'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='normal', name='dense')
    ])
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
    return model


In [ ]:
drop2_hist, drop2_evas = kfold_train(create_drop_model_2, 'dropout2_lstm', batch_size=256, epochs=30, shuffle=False, verbose=1)

In [82]:
process_results('dropout2_lstm')

mean
 acc         0.742694
loss        0.498521
val_acc     0.734458
val_loss    0.580971
dtype: float64
std
 acc         0.104400
loss        0.134317
val_acc     0.098319
val_loss    0.286717
dtype: float64


media
 acc       0.810000
f1        0.835971
mse       0.190000
recall    0.839154
dtype: float64
std
 acc       0.013071
f1        0.010510
mse       0.013071
recall    0.031162
dtype: float64


In [58]:
drop2_pred = compute_full_model(create_simpler_model, 'dropout2_lstm', batch_size=256, epochs=30, shuffle=False, verbose=1)
compute_metrics(drop2_pred, y_test)

Train on 3290 samples
Epoch 1/30
3290/3290 [==============================] - 7s 2ms/sample - loss: 0.6927 - acc: 0.5757
Epoch 2/30
3290/3290 [==============================] - 3s 977us/sample - loss: 0.6914 - acc: 0.5763
Epoch 3/30
3290/3290 [==============================] - 3s 966us/sample - loss: 0.6767 - acc: 0.6204
Epoch 4/30
3290/3290 [==============================] - 3s 975us/sample - loss: 0.6397 - acc: 0.6875
Epoch 5/30
3290/3290 [==============================] - 3s 977us/sample - loss: 0.6148 - acc: 0.7155
Epoch 6/30
3290/3290 [==============================] - 3s 977us/sample - loss: 0.5666 - acc: 0.7350
Epoch 7/30
3290/3290 [==============================] - 3s 979us/sample - loss: 0.5293 - acc: 0.7635
Epoch 8/30
3290/3290 [==============================] - 3s 964us/sample - loss: 0.4959 - acc: 0.7736
Epoch 9/30
3290/3290 [==============================] - 3s 976us/sample - loss: 0.4864 - acc: 0.7726
Epoch 10/30
3290/3290 [==============================] - 3s 976us/sampl

{'mse': 0.18156028368794327,
 'recall': 0.8486352357320099,
 'f1': 0.8423645320197044,
 'acc': 0.8184397163120567}

Con un dropout de 0.2 se mejora la media de accuracy en 2 puntos y el f1 en casi 2 puntos también, sin embargo en el conjunto de test se reduce notablemente el resultado obtenido

#### Batch Normalization

In [59]:
def create_bn_model():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(10, kernel_initializer='normal', activation='relu', input_shape=(28, 300), name='lstm'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='normal', name='dense')
    ])
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
    return model

In [89]:
bn_hist, bn_evas = kfold_train(create_bn_model, 'bn_lstm', batch_size=256, epochs=30, shuffle=False, verbose=1)

logs/DL/bn_lstm/kfold1
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 [==============================] - 2s 729us/sample - loss: 0.6921 - acc: 0.5772 - val_loss: 0.6914 - val_acc: 0.6049
Epoch 2/30
2961/2961 [==============================] - 1s 181us/sample - loss: 0.6841 - acc: 0.6295 - val_loss: 0.6895 - val_acc: 0.6018
Epoch 3/30
2961/2961 [==============================] - 1s 190us/sample - loss: 0.6674 - acc: 0.6515 - val_loss: 0.6862 - val_acc: 0.6079
Epoch 4/30
2961/2961 [==============================] - 1s 186us/sample - loss: 0.6464 - acc: 0.6852 - val_loss: 0.6816 - val_acc: 0.6231
Epoch 5/30
2961/2961 [==============================] - 1s 181us/sample - loss: 0.6213 - acc: 0.7042 - val_loss: 0.6726 - val_acc: 0.6778
Epoch 6/30
2961/2961 [==============================] - 1s 189us/sample - loss: 0.5901 - acc: 0.7322 - val_loss: 0.6484 - val_acc: 0.7325
Epoch 7/30
2961/2961 [==============================] - 1s 180us/sample - loss: 0.5481 - acc: 0.7616 -

2961/2961 [==============================] - 2s 596us/sample - loss: 0.2484 - acc: 0.9220 - val_loss: 0.4816 - val_acc: 0.8298
Epoch 30/30
2961/2961 [==============================] - 2s 584us/sample - loss: 0.2435 - acc: 0.9193 - val_loss: 0.4780 - val_acc: 0.8298
logs/DL/bn_lstm/kfold3
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 [==============================] - 14s 5ms/sample - loss: 0.6923 - acc: 0.5961 - val_loss: 0.6930 - val_acc: 0.5076
Epoch 2/30
2961/2961 [==============================] - 4s 1ms/sample - loss: 0.6877 - acc: 0.6366 - val_loss: 0.6923 - val_acc: 0.5198
Epoch 3/30
2961/2961 [==============================] - 4s 1ms/sample - loss: 0.6740 - acc: 0.6589 - val_loss: 0.6860 - val_acc: 0.5927
Epoch 4/30
2961/2961 [==============================] - 4s 1ms/sample - loss: 0.6512 - acc: 0.6906 - val_loss: 0.6760 - val_acc: 0.5988
Epoch 5/30
2961/2961 [==============================] - 4s 1ms/sample - loss: 0.6322 - acc: 0.6981 - val_loss: 0.6475 -

Epoch 28/30
2961/2961 [==============================] - 1s 177us/sample - loss: 0.2379 - acc: 0.9203 - val_loss: 0.4197 - val_acc: 0.8176
Epoch 29/30
2961/2961 [==============================] - 1s 177us/sample - loss: 0.2291 - acc: 0.9260 - val_loss: 0.4207 - val_acc: 0.8176
Epoch 30/30
2961/2961 [==============================] - 1s 179us/sample - loss: 0.2355 - acc: 0.9237 - val_loss: 0.4279 - val_acc: 0.8237
logs/DL/bn_lstm/kfold5
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 [==============================] - 2s 835us/sample - loss: 0.6921 - acc: 0.5741 - val_loss: 0.6918 - val_acc: 0.5897
Epoch 2/30
2961/2961 [==============================] - 1s 177us/sample - loss: 0.6844 - acc: 0.6305 - val_loss: 0.6897 - val_acc: 0.6079
Epoch 3/30
2961/2961 [==============================] - 1s 176us/sample - loss: 0.6673 - acc: 0.6528 - val_loss: 0.6740 - val_acc: 0.6869
Epoch 4/30
2961/2961 [==============================] - 1s 180us/sample - loss: 0.6440 - acc: 0.687

Epoch 25/30
2961/2961 [==============================] - 4s 1ms/sample - loss: 0.2692 - acc: 0.8967 - val_loss: 0.4464 - val_acc: 0.8207
Epoch 26/30
2961/2961 [==============================] - 4s 1ms/sample - loss: 0.2606 - acc: 0.8970 - val_loss: 0.5261 - val_acc: 0.8207
Epoch 27/30
2961/2961 [==============================] - 3s 1ms/sample - loss: 0.2503 - acc: 0.9004 - val_loss: 0.6515 - val_acc: 0.8267
Epoch 28/30
2961/2961 [==============================] - 4s 1ms/sample - loss: 0.2469 - acc: 0.9037 - val_loss: 0.5788 - val_acc: 0.8237
Epoch 29/30
2961/2961 [==============================] - 4s 1ms/sample - loss: 0.2406 - acc: 0.9098 - val_loss: 0.6564 - val_acc: 0.8207
Epoch 30/30
2961/2961 [==============================] - 3s 1ms/sample - loss: 0.2346 - acc: 0.9112 - val_loss: 0.7031 - val_acc: 0.8237
logs/DL/bn_lstm/kfold9
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 [==============================] - 18s 6ms/sample - loss: 0.6919 - acc: 0.5930 - val_lo

Epoch 24/30
2961/2961 [==============================] - 4s 1ms/sample - loss: 0.2730 - acc: 0.8892 - val_loss: 0.4230 - val_acc: 0.8298
Epoch 25/30
2961/2961 [==============================] - 4s 1ms/sample - loss: 0.2524 - acc: 0.8943 - val_loss: 0.4456 - val_acc: 0.8237
Epoch 26/30
2961/2961 [==============================] - 4s 1ms/sample - loss: 0.2344 - acc: 0.9092 - val_loss: 0.4418 - val_acc: 0.8328
Epoch 27/30
2961/2961 [==============================] - 4s 1ms/sample - loss: 0.2178 - acc: 0.9173 - val_loss: 0.5197 - val_acc: 0.8298
Epoch 28/30
2961/2961 [==============================] - 4s 1ms/sample - loss: 0.2178 - acc: 0.9220 - val_loss: 0.5556 - val_acc: 0.8237
Epoch 29/30
2961/2961 [==============================] - 4s 1ms/sample - loss: 0.2068 - acc: 0.9243 - val_loss: 0.5326 - val_acc: 0.8267
Epoch 30/30
2961/2961 [==============================] - 4s 1ms/sample - loss: 0.2325 - acc: 0.9162 - val_loss: 0.4727 - val_acc: 0.8024


In [90]:
process_results('bn_lstm')

mean
 acc         0.824369
loss        0.410353
val_acc     0.777518
val_loss    0.522213
dtype: float64
std
 acc         0.093273
loss        0.146759
val_acc     0.078857
val_loss    0.095072
dtype: float64


media
 acc       0.812270
f1        0.838702
mse       0.187730
recall    0.838630
dtype: float64
std
 acc       0.014353
f1        0.015013
mse       0.014353
recall    0.042387
dtype: float64


In [60]:
bn_pred = compute_full_model(create_bn_model, 'bn_lstm', batch_size=256, epochs=30, shuffle=False, verbose=1)
compute_metrics(bn_pred, y_test)

Train on 3290 samples
Epoch 1/30
3290/3290 [==============================] - 1s 381us/sample - loss: 0.6918 - acc: 0.5945
Epoch 2/30
3290/3290 [==============================] - 1s 152us/sample - loss: 0.6827 - acc: 0.6368
Epoch 3/30
3290/3290 [==============================] - 1s 153us/sample - loss: 0.6672 - acc: 0.6696
Epoch 4/30
3290/3290 [==============================] - 1s 153us/sample - loss: 0.6421 - acc: 0.6948
Epoch 5/30
3290/3290 [==============================] - 0s 150us/sample - loss: 0.6039 - acc: 0.7298
Epoch 6/30
3290/3290 [==============================] - 0s 152us/sample - loss: 0.5607 - acc: 0.7650
Epoch 7/30
3290/3290 [==============================] - 1s 153us/sample - loss: 0.5224 - acc: 0.7839
Epoch 8/30
3290/3290 [==============================] - 0s 151us/sample - loss: 0.4787 - acc: 0.8137
Epoch 9/30
3290/3290 [==============================] - 1s 154us/sample - loss: 0.4484 - acc: 0.8365
Epoch 10/30
3290/3290 [==============================] - 0s 152us/sam

{'mse': 0.1773049645390071,
 'recall': 0.8294663573085846,
 'f1': 0.851190476190476,
 'acc': 0.8226950354609929}

#### Weights initialization

Probamos la inicialización de pesos por el algoritmo Xavier (glorot_normal)

In [61]:
def glorot_model():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(10, kernel_initializer='glorot_normal', activation='relu', input_shape=(28, 300), name='lstm'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='glorot_normal', name='dense')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [95]:
gl_hist, gl_evas = kfold_train(glorot_model, 'glorot_lstm', batch_size=256, epochs=30, shuffle=False, verbose=2)

logs/DL/glorot_lstm/kfold1
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 3s - loss: 0.6916 - acc: 0.5299 - val_loss: 0.6883 - val_acc: 0.6049
Epoch 2/30
2961/2961 - 1s - loss: 0.6832 - acc: 0.5762 - val_loss: 0.6843 - val_acc: 0.6049
Epoch 3/30
2961/2961 - 1s - loss: 0.6683 - acc: 0.6339 - val_loss: 0.6803 - val_acc: 0.6049
Epoch 4/30
2961/2961 - 1s - loss: 0.6438 - acc: 0.6562 - val_loss: 0.6742 - val_acc: 0.6079
Epoch 5/30
2961/2961 - 1s - loss: 0.6165 - acc: 0.6927 - val_loss: 0.6650 - val_acc: 0.6353
Epoch 6/30
2961/2961 - 1s - loss: 0.5894 - acc: 0.7166 - val_loss: 0.6538 - val_acc: 0.6717
Epoch 7/30
2961/2961 - 1s - loss: 0.5479 - acc: 0.7497 - val_loss: 0.6334 - val_acc: 0.7143
Epoch 8/30
2961/2961 - 1s - loss: 0.4968 - acc: 0.7812 - val_loss: 0.6056 - val_acc: 0.7356
Epoch 9/30
2961/2961 - 1s - loss: 0.4435 - acc: 0.8085 - val_loss: 0.5726 - val_acc: 0.7599
Epoch 10/30
2961/2961 - 1s - loss: 0.4054 - acc: 0.8278 - val_loss: 0.5456 - val_acc: 0.7812
Epoch

Epoch 27/30
2961/2961 - 3s - loss: 0.2708 - acc: 0.8858 - val_loss: 0.5587 - val_acc: 0.8237
Epoch 28/30
2961/2961 - 3s - loss: 0.3236 - acc: 0.8619 - val_loss: 0.6328 - val_acc: 0.6079
Epoch 29/30
2961/2961 - 3s - loss: 0.2617 - acc: 0.8990 - val_loss: 0.5325 - val_acc: 0.6748
Epoch 30/30
2961/2961 - 3s - loss: 0.2499 - acc: 0.9017 - val_loss: 0.4368 - val_acc: 0.7994
logs/DL/glorot_lstm/kfold4
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 19s - loss: 0.6677 - acc: 0.6282 - val_loss: 0.6866 - val_acc: 0.6018
Epoch 2/30
2961/2961 - 3s - loss: 0.6215 - acc: 0.6694 - val_loss: 0.6776 - val_acc: 0.6079
Epoch 3/30
2961/2961 - 3s - loss: 0.5726 - acc: 0.7163 - val_loss: 0.6691 - val_acc: 0.6383
Epoch 4/30
2961/2961 - 3s - loss: 0.5384 - acc: 0.7295 - val_loss: 0.6661 - val_acc: 0.6170
Epoch 5/30
2961/2961 - 3s - loss: 0.5063 - acc: 0.7552 - val_loss: 0.6555 - val_acc: 0.6991
Epoch 6/30
2961/2961 - 3s - loss: 0.4815 - acc: 0.7649 - val_loss: 0.6473 - val_acc: 0.7264
E

Epoch 23/30
2961/2961 - 3s - loss: 0.2004 - acc: 0.9392 - val_loss: 0.4840 - val_acc: 0.8267
Epoch 24/30
2961/2961 - 3s - loss: 0.1863 - acc: 0.9416 - val_loss: 0.4919 - val_acc: 0.8085
Epoch 25/30
2961/2961 - 3s - loss: 0.1944 - acc: 0.9379 - val_loss: 0.4870 - val_acc: 0.8267
Epoch 26/30
2961/2961 - 3s - loss: 0.1861 - acc: 0.9443 - val_loss: 0.4743 - val_acc: 0.8267
Epoch 27/30
2961/2961 - 3s - loss: 0.1722 - acc: 0.9514 - val_loss: 0.4899 - val_acc: 0.8267
Epoch 28/30
2961/2961 - 3s - loss: 0.1548 - acc: 0.9618 - val_loss: 0.4792 - val_acc: 0.8389
Epoch 29/30
2961/2961 - 3s - loss: 0.1470 - acc: 0.9639 - val_loss: 0.4962 - val_acc: 0.8267
Epoch 30/30
2961/2961 - 3s - loss: 0.1461 - acc: 0.9645 - val_loss: 0.5081 - val_acc: 0.8267
logs/DL/glorot_lstm/kfold7
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 20s - loss: 0.6887 - acc: 0.5795 - val_loss: 0.6904 - val_acc: 0.5836
Epoch 2/30
2961/2961 - 3s - loss: 0.6694 - acc: 0.6413 - val_loss: 0.6866 - val_acc: 0.58

Epoch 19/30
2961/2961 - 3s - loss: 0.3630 - acc: 0.8490 - val_loss: 0.4983 - val_acc: 0.7842
Epoch 20/30
2961/2961 - 3s - loss: 0.3541 - acc: 0.8514 - val_loss: 0.5025 - val_acc: 0.7629
Epoch 21/30
2961/2961 - 3s - loss: 0.3533 - acc: 0.8551 - val_loss: 0.5000 - val_acc: 0.7629
Epoch 22/30
2961/2961 - 3s - loss: 0.3456 - acc: 0.8582 - val_loss: 0.4922 - val_acc: 0.7660
Epoch 23/30
2961/2961 - 3s - loss: 0.3423 - acc: 0.8595 - val_loss: 0.4955 - val_acc: 0.7781
Epoch 24/30
2961/2961 - 3s - loss: 0.3376 - acc: 0.8598 - val_loss: 0.4949 - val_acc: 0.7781
Epoch 25/30
2961/2961 - 3s - loss: 0.3315 - acc: 0.8632 - val_loss: 0.4960 - val_acc: 0.7781
Epoch 26/30
2961/2961 - 3s - loss: 0.3235 - acc: 0.8632 - val_loss: 0.4992 - val_acc: 0.7903
Epoch 27/30
2961/2961 - 3s - loss: 0.3085 - acc: 0.8781 - val_loss: 0.5115 - val_acc: 0.7903
Epoch 28/30
2961/2961 - 3s - loss: 0.3090 - acc: 0.8798 - val_loss: 0.5176 - val_acc: 0.7872
Epoch 29/30
2961/2961 - 3s - loss: 0.2933 - acc: 0.8804 - val_loss: 0.

In [96]:
process_results('glorot_lstm')

mean
 acc         0.830993
loss        0.376538
val_acc     0.754975
val_loss    0.531392
dtype: float64
std
 acc         0.093862
loss        0.143613
val_acc     0.085672
val_loss    0.090375
dtype: float64


media
 acc       0.810355
f1        0.834371
mse       0.189645
recall    0.846984
dtype: float64
std
 acc       0.012992
f1        0.017363
mse       0.012992
recall    0.040338
dtype: float64


In [62]:
gl_pred = compute_full_model(create_bn_model, 'glorot_lstm', batch_size=256, epochs=30, shuffle=False, verbose=1)
compute_metrics(gl_pred, y_test)

Train on 3290 samples
Epoch 1/30
3290/3290 [==============================] - 4s 1ms/sample - loss: 0.6896 - acc: 0.6201
Epoch 2/30
3290/3290 [==============================] - 2s 523us/sample - loss: 0.6725 - acc: 0.6432
Epoch 3/30
3290/3290 [==============================] - 2s 517us/sample - loss: 0.6413 - acc: 0.6903
Epoch 4/30
3290/3290 [==============================] - 2s 595us/sample - loss: 0.6101 - acc: 0.7307
Epoch 5/30
3290/3290 [==============================] - 2s 618us/sample - loss: 0.5958 - acc: 0.7252
Epoch 6/30
3290/3290 [==============================] - 2s 693us/sample - loss: 0.5830 - acc: 0.7340
Epoch 7/30
3290/3290 [==============================] - 3s 944us/sample - loss: 0.5796 - acc: 0.7204
Epoch 8/30
3290/3290 [==============================] - 3s 994us/sample - loss: 0.5490 - acc: 0.7514
Epoch 9/30
3290/3290 [==============================] - 3s 998us/sample - loss: 0.5426 - acc: 0.7532
Epoch 10/30
3290/3290 [==============================] - 3s 1ms/sample 

{'mse': 0.18439716312056736,
 'recall': 0.805921052631579,
 'f1': 0.8497109826589596,
 'acc': 0.8156028368794326}

In [63]:
def glorot_model_wo_bn():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(10, kernel_initializer='glorot_normal', activation='relu', input_shape=(28, 300), name='lstm'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='glorot_normal', name='dense')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [99]:
gl__wobn_hist, gl_wobn_evas = kfold_train(glorot_model_wo_bn, 'glorot__wobn_lstm', batch_size=256, epochs=30, shuffle=False, verbose=2)

logs/DL/glorot__wobn_lstm/kfold1
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 22s - loss: 0.6926 - acc: 0.5745 - val_loss: 0.6908 - val_acc: 0.6049
Epoch 2/30
2961/2961 - 3s - loss: 0.6909 - acc: 0.5731 - val_loss: 0.6885 - val_acc: 0.6049
Epoch 3/30
2961/2961 - 3s - loss: 0.6893 - acc: 0.5731 - val_loss: 0.6861 - val_acc: 0.6049
Epoch 4/30
2961/2961 - 3s - loss: 0.6876 - acc: 0.5731 - val_loss: 0.6834 - val_acc: 0.6049
Epoch 5/30
2961/2961 - 3s - loss: 0.6850 - acc: 0.5731 - val_loss: 0.6797 - val_acc: 0.6049
Epoch 6/30
2961/2961 - 3s - loss: 0.6814 - acc: 0.5738 - val_loss: 0.6740 - val_acc: 0.6049
Epoch 7/30
2961/2961 - 3s - loss: 0.6787 - acc: 0.5738 - val_loss: 0.6668 - val_acc: 0.6049
Epoch 8/30
2961/2961 - 3s - loss: 0.6717 - acc: 0.5751 - val_loss: 0.6637 - val_acc: 0.6231
Epoch 9/30
2961/2961 - 3s - loss: 0.6579 - acc: 0.6346 - val_loss: 0.6405 - val_acc: 0.6778
Epoch 10/30
2961/2961 - 3s - loss: 0.6163 - acc: 0.7048 - val_loss: 0.6243 - val_acc: 0.699

Epoch 27/30
2961/2961 - 0s - loss: 0.5293 - acc: 0.7278 - val_loss: 0.5681 - val_acc: 0.6869
Epoch 28/30
2961/2961 - 0s - loss: 0.5253 - acc: 0.7302 - val_loss: 0.5625 - val_acc: 0.6900
Epoch 29/30
2961/2961 - 0s - loss: 0.5255 - acc: 0.7295 - val_loss: 0.5537 - val_acc: 0.6960
Epoch 30/30
2961/2961 - 1s - loss: 0.5163 - acc: 0.7302 - val_loss: 0.5426 - val_acc: 0.7021
logs/DL/glorot__wobn_lstm/kfold4
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 23s - loss: 0.6923 - acc: 0.5748 - val_loss: 0.6904 - val_acc: 0.5866
Epoch 2/30
2961/2961 - 3s - loss: 0.6898 - acc: 0.5751 - val_loss: 0.6871 - val_acc: 0.5866
Epoch 3/30
2961/2961 - 3s - loss: 0.6858 - acc: 0.5751 - val_loss: 0.6779 - val_acc: 0.5866
Epoch 4/30
2961/2961 - 3s - loss: 0.6658 - acc: 0.5751 - val_loss: 0.6608 - val_acc: 0.5866
Epoch 5/30
2961/2961 - 3s - loss: 0.6609 - acc: 0.5755 - val_loss: 0.6266 - val_acc: 0.5866
Epoch 6/30
2961/2961 - 3s - loss: 0.7149 - acc: 0.5772 - val_loss: 0.6220 - val_acc: 0.

Epoch 23/30
2961/2961 - 1s - loss: 0.3515 - acc: 0.8514 - val_loss: 0.9278 - val_acc: 0.8389
Epoch 24/30
2961/2961 - 1s - loss: 0.3404 - acc: 0.8568 - val_loss: 0.7869 - val_acc: 0.8480
Epoch 25/30
2961/2961 - 1s - loss: 0.3340 - acc: 0.8622 - val_loss: 0.8639 - val_acc: 0.8450
Epoch 26/30
2961/2961 - 1s - loss: 0.3331 - acc: 0.8548 - val_loss: 0.8873 - val_acc: 0.8419
Epoch 27/30
2961/2961 - 1s - loss: 0.3277 - acc: 0.8609 - val_loss: 0.9822 - val_acc: 0.8419
Epoch 28/30
2961/2961 - 1s - loss: 0.3226 - acc: 0.8622 - val_loss: 0.7181 - val_acc: 0.8450
Epoch 29/30
2961/2961 - 1s - loss: 0.3169 - acc: 0.8619 - val_loss: 0.8263 - val_acc: 0.8450
Epoch 30/30
2961/2961 - 1s - loss: 0.3198 - acc: 0.8625 - val_loss: 0.8272 - val_acc: 0.8450
logs/DL/glorot__wobn_lstm/kfold7
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 24s - loss: 0.6926 - acc: 0.5721 - val_loss: 0.6914 - val_acc: 0.5836
Epoch 2/30
2961/2961 - 3s - loss: 0.6910 - acc: 0.5755 - val_loss: 0.6894 - val_acc

Epoch 19/30
2961/2961 - 1s - loss: 0.3900 - acc: 0.8132 - val_loss: 0.4528 - val_acc: 0.7781
Epoch 20/30
2961/2961 - 2s - loss: 0.3999 - acc: 0.8092 - val_loss: 0.4321 - val_acc: 0.7842
Epoch 21/30
2961/2961 - 2s - loss: 0.3736 - acc: 0.8288 - val_loss: 0.4588 - val_acc: 0.7751
Epoch 22/30
2961/2961 - 2s - loss: 0.3512 - acc: 0.8382 - val_loss: 0.4408 - val_acc: 0.7781
Epoch 23/30
2961/2961 - 2s - loss: 0.3383 - acc: 0.8403 - val_loss: 0.4522 - val_acc: 0.7781
Epoch 24/30
2961/2961 - 3s - loss: 0.3307 - acc: 0.8457 - val_loss: 0.4452 - val_acc: 0.7812
Epoch 25/30
2961/2961 - 3s - loss: 0.3228 - acc: 0.8501 - val_loss: 0.4351 - val_acc: 0.7842
Epoch 26/30
2961/2961 - 3s - loss: 0.3159 - acc: 0.8534 - val_loss: 0.4430 - val_acc: 0.7872
Epoch 27/30
2961/2961 - 3s - loss: 0.3086 - acc: 0.8565 - val_loss: 0.4392 - val_acc: 0.7903
Epoch 28/30
2961/2961 - 3s - loss: 0.3094 - acc: 0.8565 - val_loss: 0.4343 - val_acc: 0.7903
Epoch 29/30
2961/2961 - 3s - loss: 0.2971 - acc: 0.8602 - val_loss: 0.

In [100]:
process_results('glorot__wobn_lstm')

mean
 acc         0.723824
loss        0.515169
val_acc     0.718045
val_loss    0.577295
dtype: float64
std
 acc         0.101979
loss        0.125273
val_acc     0.095369
val_loss    0.119863
dtype: float64


media
 acc       0.797305
f1        0.817123
mse       0.202695
recall    0.864239
dtype: float64
std
 acc       0.027642
f1        0.021130
mse       0.027642
recall    0.055857
dtype: float64


In [64]:
gl_wobn_pred = compute_full_model(glorot_model_wo_bn, 'glorot__wobn_lstm', batch_size=256, epochs=30, shuffle=False, verbose=1)
compute_metrics(gl_wobn_pred, y_test)

Train on 3290 samples
Epoch 1/30
3290/3290 [==============================] - 7s 2ms/sample - loss: 0.6924 - acc: 0.5623
Epoch 2/30
3290/3290 [==============================] - 3s 974us/sample - loss: 0.6888 - acc: 0.6073
Epoch 3/30
3290/3290 [==============================] - 3s 953us/sample - loss: 0.6723 - acc: 0.6711
Epoch 4/30
3290/3290 [==============================] - 3s 965us/sample - loss: 0.6780 - acc: 0.6264
Epoch 5/30
3290/3290 [==============================] - 3s 977us/sample - loss: 0.6688 - acc: 0.6225
Epoch 6/30
3290/3290 [==============================] - 3s 951us/sample - loss: 0.6325 - acc: 0.6973
Epoch 7/30
3290/3290 [==============================] - 3s 981us/sample - loss: 0.6498 - acc: 0.6559
Epoch 8/30
3290/3290 [==============================] - 3s 1ms/sample - loss: 0.6250 - acc: 0.6796
Epoch 9/30
3290/3290 [==============================] - 4s 1ms/sample - loss: 0.5773 - acc: 0.7353
Epoch 10/30
3290/3290 [==============================] - 3s 1ms/sample - lo

{'mse': 0.16028368794326242,
 'recall': 0.8824289405684754,
 'f1': 0.8580402010050251,
 'acc': 0.8397163120567376}

## Pruebas con topologías

### 2 lstm

In [65]:
def double_lstm_model():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(10, kernel_initializer='glorot_normal', activation='relu', return_sequences=True, input_shape=(28, 300), name='lstm'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(10, activation='relu', name='2lstm'),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='glorot_normal', name='dense')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [103]:
double_hist, double_evas = kfold_train(double_lstm_model, 'double_lstm', batch_size=256, epochs=30, shuffle=False, verbose=2)

logs/DL/double_lstm/kfold1
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 31s - loss: 0.6919 - acc: 0.5599 - val_loss: 0.6887 - val_acc: 0.6049
Epoch 2/30
2961/2961 - 5s - loss: 0.6880 - acc: 0.5751 - val_loss: 0.6838 - val_acc: 0.6201
Epoch 3/30
2961/2961 - 5s - loss: 0.6757 - acc: 0.6532 - val_loss: 0.6681 - val_acc: 0.6444
Epoch 4/30
2961/2961 - 5s - loss: 0.6558 - acc: 0.7058 - val_loss: 0.6547 - val_acc: 0.6991
Epoch 5/30
2961/2961 - 5s - loss: 0.6136 - acc: 0.7349 - val_loss: 0.6088 - val_acc: 0.7325
Epoch 6/30
2961/2961 - 5s - loss: 0.5680 - acc: 0.7511 - val_loss: 0.5793 - val_acc: 0.7447
Epoch 7/30
2961/2961 - 5s - loss: 0.5219 - acc: 0.7619 - val_loss: 0.5230 - val_acc: 0.7538
Epoch 8/30
2961/2961 - 5s - loss: 0.5086 - acc: 0.7734 - val_loss: 0.5085 - val_acc: 0.7508
Epoch 9/30
2961/2961 - 5s - loss: 0.4621 - acc: 0.7842 - val_loss: 0.4958 - val_acc: 0.7660
Epoch 10/30
2961/2961 - 5s - loss: 0.4368 - acc: 0.7937 - val_loss: 0.4804 - val_acc: 0.7720
Epoc

Epoch 27/30
2961/2961 - 5s - loss: 0.6192 - acc: 0.5839 - val_loss: 0.6372 - val_acc: 0.5076
Epoch 28/30
2961/2961 - 5s - loss: 0.6131 - acc: 0.5839 - val_loss: 0.6330 - val_acc: 0.5076
Epoch 29/30
2961/2961 - 5s - loss: 0.6083 - acc: 0.5839 - val_loss: 0.6280 - val_acc: 0.5076
Epoch 30/30
2961/2961 - 4s - loss: 0.6023 - acc: 0.5839 - val_loss: 0.6224 - val_acc: 0.5076
logs/DL/double_lstm/kfold4
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 6s - loss: 0.6919 - acc: 0.5751 - val_loss: 0.6884 - val_acc: 0.5866
Epoch 2/30
2961/2961 - 1s - loss: 0.6873 - acc: 0.5751 - val_loss: 0.6815 - val_acc: 0.5866
Epoch 3/30
2961/2961 - 1s - loss: 0.6837 - acc: 0.5751 - val_loss: 0.6692 - val_acc: 0.5866
Epoch 4/30
2961/2961 - 1s - loss: 0.6734 - acc: 0.5751 - val_loss: 0.6690 - val_acc: 0.5866
Epoch 5/30
2961/2961 - 1s - loss: 0.6664 - acc: 0.5751 - val_loss: 0.6436 - val_acc: 0.5866
Epoch 6/30
2961/2961 - 1s - loss: 0.6379 - acc: 0.5751 - val_loss: 0.6175 - val_acc: 0.5897
Ep

Epoch 23/30
2961/2961 - 5s - loss: 0.3692 - acc: 0.8460 - val_loss: 0.4048 - val_acc: 0.8146
Epoch 24/30
2961/2961 - 5s - loss: 0.3641 - acc: 0.8440 - val_loss: 0.4028 - val_acc: 0.8116
Epoch 25/30
2961/2961 - 5s - loss: 0.3559 - acc: 0.8470 - val_loss: 0.4025 - val_acc: 0.8176
Epoch 26/30
2961/2961 - 5s - loss: 0.3513 - acc: 0.8460 - val_loss: 0.4022 - val_acc: 0.8237
Epoch 27/30
2961/2961 - 5s - loss: 0.3452 - acc: 0.8521 - val_loss: 0.4013 - val_acc: 0.8237
Epoch 28/30
2961/2961 - 5s - loss: 0.3373 - acc: 0.8598 - val_loss: 0.3959 - val_acc: 0.8267
Epoch 29/30
2961/2961 - 5s - loss: 0.3274 - acc: 0.8666 - val_loss: 0.3965 - val_acc: 0.8207
Epoch 30/30
2961/2961 - 5s - loss: 0.3189 - acc: 0.8696 - val_loss: 0.3966 - val_acc: 0.8267
logs/DL/double_lstm/kfold7
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 6s - loss: 0.6919 - acc: 0.5741 - val_loss: 0.6885 - val_acc: 0.5836
Epoch 2/30
2961/2961 - 1s - loss: 0.6856 - acc: 0.5755 - val_loss: 0.6671 - val_acc: 0.583

Epoch 19/30
2961/2961 - 5s - loss: 0.5816 - acc: 0.6727 - val_loss: 0.5823 - val_acc: 0.6717
Epoch 20/30
2961/2961 - 5s - loss: 0.5697 - acc: 0.6700 - val_loss: 0.5828 - val_acc: 0.6657
Epoch 21/30
2961/2961 - 5s - loss: 0.5755 - acc: 0.6964 - val_loss: 0.5746 - val_acc: 0.7325
Epoch 22/30
2961/2961 - 5s - loss: 0.5661 - acc: 0.7052 - val_loss: 0.5836 - val_acc: 0.7234
Epoch 23/30
2961/2961 - 5s - loss: 0.5591 - acc: 0.7069 - val_loss: 0.5595 - val_acc: 0.7204
Epoch 24/30
2961/2961 - 5s - loss: 0.5555 - acc: 0.7194 - val_loss: 0.5608 - val_acc: 0.7508
Epoch 25/30
2961/2961 - 5s - loss: 0.5493 - acc: 0.7278 - val_loss: 0.5595 - val_acc: 0.7599
Epoch 26/30
2961/2961 - 5s - loss: 0.5449 - acc: 0.7366 - val_loss: 0.5529 - val_acc: 0.7629
Epoch 27/30
2961/2961 - 5s - loss: 0.5343 - acc: 0.7339 - val_loss: 0.5390 - val_acc: 0.7599
Epoch 28/30
2961/2961 - 5s - loss: 0.5363 - acc: 0.7396 - val_loss: 0.5510 - val_acc: 0.7477
Epoch 29/30
2961/2961 - 5s - loss: 0.5341 - acc: 0.7464 - val_loss: 0.

In [104]:
process_results('double_lstm')

mean
 acc         0.744781
loss        0.480423
val_acc     0.729200
val_loss    0.536301
dtype: float64
std
 acc         0.114811
loss        0.143363
val_acc     0.106530
val_loss    0.171783
dtype: float64


media
 acc       0.780993
f1        0.809816
mse       0.219007
recall    0.858894
dtype: float64
std
 acc       0.075550
f1        0.034890
mse       0.075550
recall    0.113981
dtype: float64


In [66]:
double_pred = compute_full_model(double_lstm_model, 'double_lstm', batch_size=256, epochs=30, shuffle=False, verbose=1)
compute_metrics(double_pred, y_test)

Train on 3290 samples
Epoch 1/30
3290/3290 [==============================] - 12s 4ms/sample - loss: 0.6919 - acc: 0.5650
Epoch 2/30
3290/3290 [==============================] - 6s 2ms/sample - loss: 0.6878 - acc: 0.5784
Epoch 3/30
3290/3290 [==============================] - 6s 2ms/sample - loss: 0.6710 - acc: 0.6550
Epoch 4/30
3290/3290 [==============================] - 6s 2ms/sample - loss: 0.6303 - acc: 0.6723
Epoch 5/30
3290/3290 [==============================] - 6s 2ms/sample - loss: 0.5824 - acc: 0.6897
Epoch 6/30
3290/3290 [==============================] - 6s 2ms/sample - loss: 0.5332 - acc: 0.7207
Epoch 7/30
3290/3290 [==============================] - 6s 2ms/sample - loss: 0.5118 - acc: 0.7210
Epoch 8/30
3290/3290 [==============================] - 5s 2ms/sample - loss: 0.4705 - acc: 0.7584
Epoch 9/30
3290/3290 [==============================] - 5s 2ms/sample - loss: 0.4382 - acc: 0.7729
Epoch 10/30
3290/3290 [==============================] - 1s 271us/sample - loss: 0.416

{'mse': 0.1829787234042553,
 'recall': 0.9142011834319527,
 'f1': 0.8273092369477912,
 'acc': 0.8170212765957446}

### Double lstm fine tune

### Convolucional

In [67]:
def convolutional_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape((28, 300, 1), input_shape=(28, 300), name='lstm'),
        tf.keras.layers.Conv2D(128, (4, 300), padding='same', name='conv_layer'),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(16,16), strides=None),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='normal', name='dense')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [120]:
convolutional_model().summary()

Model: "sequential_67"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (Reshape)               (None, 28, 300, 1)        0         
_________________________________________________________________
conv_layer (Conv2D)          (None, 28, 300, 128)      153728    
_________________________________________________________________
activation_7 (Activation)    (None, 28, 300, 128)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 1, 18, 128)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dropout_63 (Dropout)         (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 1)               

In [122]:
conv_hist, conv_evas = kfold_train(convolutional_model, 'convolutional', batch_size=8, epochs=30, shuffle=False, verbose=2)

logs/DL/convolutional/kfold1
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 49s - loss: 0.4657 - acc: 0.7758 - val_loss: 0.5179 - val_acc: 0.7295
Epoch 2/30
2961/2961 - 20s - loss: 0.3148 - acc: 0.8696 - val_loss: 0.4819 - val_acc: 0.7447
Epoch 3/30
2961/2961 - 20s - loss: 0.1958 - acc: 0.9331 - val_loss: 0.5921 - val_acc: 0.7416
Epoch 4/30
2961/2961 - 11s - loss: 0.1439 - acc: 0.9500 - val_loss: 0.4712 - val_acc: 0.7781
Epoch 5/30
2961/2961 - 11s - loss: 0.1272 - acc: 0.9554 - val_loss: 0.4111 - val_acc: 0.8389
Epoch 6/30
2961/2961 - 11s - loss: 0.1271 - acc: 0.9497 - val_loss: 0.4283 - val_acc: 0.8146
Epoch 7/30
2961/2961 - 11s - loss: 0.1622 - acc: 0.9365 - val_loss: 0.7083 - val_acc: 0.7416
Epoch 8/30
2961/2961 - 11s - loss: 0.1173 - acc: 0.9554 - val_loss: 0.4799 - val_acc: 0.8146
Epoch 9/30
2961/2961 - 11s - loss: 0.0748 - acc: 0.9726 - val_loss: 0.4806 - val_acc: 0.8085
Epoch 10/30
2961/2961 - 11s - loss: 0.0639 - acc: 0.9767 - val_loss: 0.5748 - val_acc: 

2961/2961 - 19s - loss: 0.0280 - acc: 0.9902 - val_loss: 0.6684 - val_acc: 0.8419
Epoch 27/30
2961/2961 - 19s - loss: 0.0163 - acc: 0.9936 - val_loss: 0.7703 - val_acc: 0.8146
Epoch 28/30
2961/2961 - 19s - loss: 0.0078 - acc: 0.9986 - val_loss: 0.7539 - val_acc: 0.8450
Epoch 29/30
2961/2961 - 19s - loss: 0.0152 - acc: 0.9939 - val_loss: 0.6779 - val_acc: 0.8541
Epoch 30/30
2961/2961 - 19s - loss: 0.0087 - acc: 0.9983 - val_loss: 0.7498 - val_acc: 0.8389
logs/DL/convolutional/kfold4
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 48s - loss: 0.4731 - acc: 0.7791 - val_loss: 0.6145 - val_acc: 0.6839
Epoch 2/30
2961/2961 - 19s - loss: 0.3133 - acc: 0.8734 - val_loss: 0.5751 - val_acc: 0.7629
Epoch 3/30
2961/2961 - 19s - loss: 0.1988 - acc: 0.9284 - val_loss: 0.7216 - val_acc: 0.7112
Epoch 4/30
2961/2961 - 19s - loss: 0.1440 - acc: 0.9517 - val_loss: 0.5647 - val_acc: 0.8085
Epoch 5/30
2961/2961 - 19s - loss: 0.1382 - acc: 0.9480 - val_loss: 0.6289 - val_acc: 0.7933
E

Epoch 22/30
2961/2961 - 19s - loss: 0.0120 - acc: 0.9976 - val_loss: 0.9253 - val_acc: 0.8146
Epoch 23/30
2961/2961 - 19s - loss: 0.0164 - acc: 0.9953 - val_loss: 0.7808 - val_acc: 0.8298
Epoch 24/30
2961/2961 - 19s - loss: 0.0156 - acc: 0.9946 - val_loss: 0.8153 - val_acc: 0.8267
Epoch 25/30
2961/2961 - 19s - loss: 0.0163 - acc: 0.9953 - val_loss: 0.8184 - val_acc: 0.8298
Epoch 26/30
2961/2961 - 19s - loss: 0.0253 - acc: 0.9909 - val_loss: 0.8760 - val_acc: 0.8267
Epoch 27/30
2961/2961 - 19s - loss: 0.0284 - acc: 0.9895 - val_loss: 0.8734 - val_acc: 0.8237
Epoch 28/30
2961/2961 - 19s - loss: 0.0262 - acc: 0.9916 - val_loss: 0.8697 - val_acc: 0.8237
Epoch 29/30
2961/2961 - 19s - loss: 0.0258 - acc: 0.9926 - val_loss: 1.0450 - val_acc: 0.8146
Epoch 30/30
2961/2961 - 19s - loss: 0.0253 - acc: 0.9902 - val_loss: 0.9358 - val_acc: 0.8359
logs/DL/convolutional/kfold7
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 49s - loss: 0.4745 - acc: 0.7744 - val_loss: 0.4669 - v

2961/2961 - 19s - loss: 0.0049 - acc: 1.0000 - val_loss: 0.7263 - val_acc: 0.8055
Epoch 18/30
2961/2961 - 19s - loss: 0.0038 - acc: 1.0000 - val_loss: 0.7566 - val_acc: 0.8055
Epoch 19/30
2961/2961 - 19s - loss: 0.0156 - acc: 0.9953 - val_loss: 0.7734 - val_acc: 0.8207
Epoch 20/30
2961/2961 - 19s - loss: 0.0973 - acc: 0.9612 - val_loss: 0.9459 - val_acc: 0.7812
Epoch 21/30
2961/2961 - 19s - loss: 0.0585 - acc: 0.9791 - val_loss: 0.7918 - val_acc: 0.8116
Epoch 22/30
2961/2961 - 19s - loss: 0.0397 - acc: 0.9868 - val_loss: 0.9162 - val_acc: 0.7994
Epoch 23/30
2961/2961 - 19s - loss: 0.0405 - acc: 0.9845 - val_loss: 1.3672 - val_acc: 0.7447
Epoch 24/30
2961/2961 - 19s - loss: 0.0289 - acc: 0.9912 - val_loss: 0.9462 - val_acc: 0.8024
Epoch 25/30
2961/2961 - 19s - loss: 0.0133 - acc: 0.9956 - val_loss: 0.9990 - val_acc: 0.8055
Epoch 26/30
2961/2961 - 19s - loss: 0.0153 - acc: 0.9953 - val_loss: 0.9747 - val_acc: 0.7994
Epoch 27/30
2961/2961 - 19s - loss: 0.0142 - acc: 0.9956 - val_loss: 0.9

In [123]:
process_results('convolutional')

mean
 acc         0.969988
loss        0.078238
val_acc     0.808744
val_loss    0.682893
dtype: float64
std
 acc         0.046071
loss        0.102944
val_acc     0.040357
val_loss    0.192932
dtype: float64


media
 acc       0.826170
f1        0.849522
mse       0.173830
recall    0.856080
dtype: float64
std
 acc       0.010933
f1        0.006497
mse       0.010933
recall    0.034780
dtype: float64


In [68]:
conv_pred = compute_full_model(convolutional_model, 'convolutional', batch_size=8, epochs=30, shuffle=False, verbose=1)
compute_metrics(conv_pred, y_test)

Train on 3290 samples
Epoch 1/30
3290/3290 [==============================] - 26s 8ms/sample - loss: 0.4668 - acc: 0.7754
Epoch 2/30
3290/3290 [==============================] - 23s 7ms/sample - loss: 0.3290 - acc: 0.8590
Epoch 3/30
3290/3290 [==============================] - 23s 7ms/sample - loss: 0.2120 - acc: 0.9237
Epoch 4/30
3290/3290 [==============================] - 23s 7ms/sample - loss: 0.1377 - acc: 0.9544
Epoch 5/30
3290/3290 [==============================] - 23s 7ms/sample - loss: 0.1029 - acc: 0.9666
Epoch 6/30
3290/3290 [==============================] - 23s 7ms/sample - loss: 0.1225 - acc: 0.9544
Epoch 7/30
3290/3290 [==============================] - 23s 7ms/sample - loss: 0.1318 - acc: 0.9450
Epoch 8/30
3290/3290 [==============================] - 23s 7ms/sample - loss: 0.1170 - acc: 0.9556
Epoch 9/30
3290/3290 [==============================] - 23s 7ms/sample - loss: 0.1051 - acc: 0.9608
Epoch 10/30
3290/3290 [==============================] - 23s 7ms/sample - loss

{'mse': 0.21063829787234042,
 'recall': 0.9363484087102177,
 'f1': 0.7901060070671378,
 'acc': 0.7893617021276595}

### Convolutional 1D

In [69]:
def convolutional1d_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(4, 300, input_shape=(28, 300), padding='same', name='conv_layer'),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.MaxPooling1D(pool_size=(4), strides=None),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='normal', name='dense')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [144]:
convolutional1d_model().summary()

Model: "sequential_88"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_layer (Conv1D)          (None, 28, 4)             360004    
_________________________________________________________________
activation_28 (Activation)   (None, 28, 4)             0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 7, 4)              0         
_________________________________________________________________
flatten_28 (Flatten)         (None, 28)                0         
_________________________________________________________________
dropout_84 (Dropout)         (None, 28)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 29        
Total params: 360,033
Trainable params: 360,033
Non-trainable params: 0
_______________________________________________

In [145]:
conv1d_hist, conv1d_evas = kfold_train(convolutional1d_model, 'convolutional1d', batch_size=8, epochs=30, shuffle=False, verbose=2)

logs/DL/convolutional1d/kfold1
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 11s - loss: 0.5054 - acc: 0.7548 - val_loss: 0.4201 - val_acc: 0.7812
Epoch 2/30
2961/2961 - 6s - loss: 0.3594 - acc: 0.8453 - val_loss: 0.3830 - val_acc: 0.8085
Epoch 3/30
2961/2961 - 7s - loss: 0.2877 - acc: 0.8835 - val_loss: 0.3895 - val_acc: 0.8116
Epoch 4/30
2961/2961 - 11s - loss: 0.2248 - acc: 0.9173 - val_loss: 0.3807 - val_acc: 0.8085
Epoch 5/30
2961/2961 - 14s - loss: 0.1732 - acc: 0.9443 - val_loss: 0.3870 - val_acc: 0.8298
Epoch 6/30
2961/2961 - 15s - loss: 0.1288 - acc: 0.9588 - val_loss: 0.4125 - val_acc: 0.8328
Epoch 7/30
2961/2961 - 14s - loss: 0.1067 - acc: 0.9706 - val_loss: 0.4311 - val_acc: 0.8146
Epoch 8/30
2961/2961 - 13s - loss: 0.0738 - acc: 0.9821 - val_loss: 0.4210 - val_acc: 0.8328
Epoch 9/30
2961/2961 - 13s - loss: 0.0600 - acc: 0.9872 - val_loss: 0.4887 - val_acc: 0.8207
Epoch 10/30
2961/2961 - 14s - loss: 0.0511 - acc: 0.9872 - val_loss: 0.4994 - val_acc: 

Epoch 27/30
2961/2961 - 13s - loss: 0.0130 - acc: 0.9946 - val_loss: 0.7507 - val_acc: 0.8511
Epoch 28/30
2961/2961 - 13s - loss: 0.0084 - acc: 0.9986 - val_loss: 0.8525 - val_acc: 0.8419
Epoch 29/30
2961/2961 - 13s - loss: 0.0078 - acc: 0.9983 - val_loss: 0.8023 - val_acc: 0.8480
Epoch 30/30
2961/2961 - 13s - loss: 0.0092 - acc: 0.9973 - val_loss: 0.7130 - val_acc: 0.8602
logs/DL/convolutional1d/kfold4
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 44s - loss: 0.5186 - acc: 0.7396 - val_loss: 0.4179 - val_acc: 0.8298
Epoch 2/30
2961/2961 - 13s - loss: 0.3567 - acc: 0.8460 - val_loss: 0.4120 - val_acc: 0.8419
Epoch 3/30
2961/2961 - 13s - loss: 0.2779 - acc: 0.8879 - val_loss: 0.4394 - val_acc: 0.8389
Epoch 4/30
2961/2961 - 13s - loss: 0.2090 - acc: 0.9200 - val_loss: 0.4718 - val_acc: 0.8419
Epoch 5/30
2961/2961 - 13s - loss: 0.1545 - acc: 0.9466 - val_loss: 0.5019 - val_acc: 0.8237
Epoch 6/30
2961/2961 - 9s - loss: 0.1125 - acc: 0.9686 - val_loss: 0.5493 - val_a

Epoch 23/30
2961/2961 - 6s - loss: 0.0126 - acc: 0.9973 - val_loss: 0.8165 - val_acc: 0.8207
Epoch 24/30
2961/2961 - 6s - loss: 0.0168 - acc: 0.9949 - val_loss: 0.8836 - val_acc: 0.8237
Epoch 25/30
2961/2961 - 6s - loss: 0.0148 - acc: 0.9959 - val_loss: 0.9344 - val_acc: 0.8146
Epoch 26/30
2961/2961 - 6s - loss: 0.0118 - acc: 0.9963 - val_loss: 0.9128 - val_acc: 0.8146
Epoch 27/30
2961/2961 - 6s - loss: 0.0103 - acc: 0.9963 - val_loss: 0.9007 - val_acc: 0.8298
Epoch 28/30
2961/2961 - 6s - loss: 0.0090 - acc: 0.9976 - val_loss: 0.9640 - val_acc: 0.8207
Epoch 29/30
2961/2961 - 6s - loss: 0.0109 - acc: 0.9970 - val_loss: 1.1942 - val_acc: 0.8146
Epoch 30/30
2961/2961 - 6s - loss: 0.0073 - acc: 0.9983 - val_loss: 1.0200 - val_acc: 0.8237
logs/DL/convolutional1d/kfold7
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 19s - loss: 0.5398 - acc: 0.7275 - val_loss: 0.4267 - val_acc: 0.7812
Epoch 2/30
2961/2961 - 11s - loss: 0.3742 - acc: 0.8376 - val_loss: 0.3984 - val_acc:

Epoch 19/30
2961/2961 - 13s - loss: 0.0183 - acc: 0.9946 - val_loss: 0.9231 - val_acc: 0.7812
Epoch 20/30
2961/2961 - 13s - loss: 0.0144 - acc: 0.9953 - val_loss: 0.9025 - val_acc: 0.7629
Epoch 21/30
2961/2961 - 13s - loss: 0.0220 - acc: 0.9936 - val_loss: 0.9706 - val_acc: 0.7751
Epoch 22/30
2961/2961 - 13s - loss: 0.0151 - acc: 0.9959 - val_loss: 0.9337 - val_acc: 0.7933
Epoch 23/30
2961/2961 - 13s - loss: 0.0114 - acc: 0.9966 - val_loss: 0.9772 - val_acc: 0.7812
Epoch 24/30
2961/2961 - 11s - loss: 0.0087 - acc: 0.9986 - val_loss: 1.1341 - val_acc: 0.7720
Epoch 25/30
2961/2961 - 6s - loss: 0.0103 - acc: 0.9973 - val_loss: 1.2600 - val_acc: 0.7538
Epoch 26/30
2961/2961 - 6s - loss: 0.0081 - acc: 0.9976 - val_loss: 1.2025 - val_acc: 0.7751
Epoch 27/30
2961/2961 - 6s - loss: 0.0083 - acc: 0.9986 - val_loss: 1.0707 - val_acc: 0.8116
Epoch 28/30
2961/2961 - 6s - loss: 0.0059 - acc: 0.9990 - val_loss: 1.1481 - val_acc: 0.7933
Epoch 29/30
2961/2961 - 6s - loss: 0.0071 - acc: 0.9980 - val_lo

In [146]:
process_results('convolutional1d')

mean
 acc         0.970137
loss        0.079617
val_acc     0.815268
val_loss    0.664958
dtype: float64
std
 acc         0.057874
loss        0.122522
val_acc     0.022652
val_loss    0.202961
dtype: float64


media
 acc       0.819362
f1        0.839427
mse       0.180638
recall    0.867145
dtype: float64
std
 acc       0.009101
f1        0.010347
mse       0.009101
recall    0.023552
dtype: float64


In [70]:
conv1d_pred = compute_full_model(convolutional1d_model, 'convolutional1d', batch_size=8, epochs=30, shuffle=False, verbose=1)
compute_metrics(conv1d_pred, y_test)

Train on 3290 samples
Epoch 1/30
3290/3290 [==============================] - 16s 5ms/sample - loss: 0.5088 - acc: 0.7331
Epoch 2/30
3290/3290 [==============================] - 14s 4ms/sample - loss: 0.3623 - acc: 0.8477
Epoch 3/30
3290/3290 [==============================] - 14s 4ms/sample - loss: 0.2919 - acc: 0.8790
Epoch 4/30
3290/3290 [==============================] - 13s 4ms/sample - loss: 0.2386 - acc: 0.9085
Epoch 5/30
3290/3290 [==============================] - 13s 4ms/sample - loss: 0.1884 - acc: 0.9353
Epoch 6/30
3290/3290 [==============================] - 14s 4ms/sample - loss: 0.1336 - acc: 0.9584
Epoch 7/30
3290/3290 [==============================] - 13s 4ms/sample - loss: 0.1050 - acc: 0.9699
Epoch 8/30
3290/3290 [==============================] - 13s 4ms/sample - loss: 0.0821 - acc: 0.9766
Epoch 9/30
3290/3290 [==============================] - 13s 4ms/sample - loss: 0.0610 - acc: 0.98391s - l
Epoch 10/30
3290/3290 [==============================] - 14s 4ms/sample 

{'mse': 0.18581560283687942,
 'recall': 0.8021739130434783,
 'f1': 0.8492520138089757,
 'acc': 0.8141843971631205}

### Bidirectional

In [71]:
def bidirectional_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(10), input_shape=(28, 300)),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [156]:
bidi_hist, bidi_evas = kfold_train(bidirectional_model, 'bidirectional', batch_size=8, epochs=30, shuffle=False, verbose=2)

logs/DL/bidirectional/kfold1
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 90s - loss: 0.5217 - acc: 0.7396 - val_loss: 0.4384 - val_acc: 0.7964
Epoch 2/30
2961/2961 - 150s - loss: 0.4033 - acc: 0.8291 - val_loss: 0.4084 - val_acc: 0.8146
Epoch 3/30
2961/2961 - 22s - loss: 0.3733 - acc: 0.8467 - val_loss: 0.3993 - val_acc: 0.8146
Epoch 4/30
2961/2961 - 134s - loss: 0.3509 - acc: 0.8561 - val_loss: 0.3929 - val_acc: 0.8146
Epoch 5/30
2961/2961 - 146s - loss: 0.3321 - acc: 0.8666 - val_loss: 0.3941 - val_acc: 0.8207
Epoch 6/30
2961/2961 - 123s - loss: 0.3046 - acc: 0.8835 - val_loss: 0.4039 - val_acc: 0.8116
Epoch 7/30
2961/2961 - 22s - loss: 0.2885 - acc: 0.8896 - val_loss: 0.4001 - val_acc: 0.8085
Epoch 8/30
2961/2961 - 131s - loss: 0.2603 - acc: 0.9054 - val_loss: 0.4085 - val_acc: 0.8146
Epoch 9/30
2961/2961 - 142s - loss: 0.2407 - acc: 0.9129 - val_loss: 0.4280 - val_acc: 0.8055
Epoch 10/30
2961/2961 - 98s - loss: 0.2251 - acc: 0.9169 - val_loss: 0.4259 - val

Epoch 26/30
2961/2961 - 58s - loss: 0.0834 - acc: 0.9693 - val_loss: 0.5406 - val_acc: 0.8176
Epoch 27/30
2961/2961 - 70s - loss: 0.0899 - acc: 0.9710 - val_loss: 0.6188 - val_acc: 0.8267
Epoch 28/30
2961/2961 - 142s - loss: 0.0706 - acc: 0.9777 - val_loss: 0.6571 - val_acc: 0.8267
Epoch 29/30
2961/2961 - 142s - loss: 0.0676 - acc: 0.9794 - val_loss: 0.6668 - val_acc: 0.8267
Epoch 30/30
2961/2961 - 106s - loss: 0.0567 - acc: 0.9841 - val_loss: 0.6701 - val_acc: 0.8267
logs/DL/bidirectional/kfold4
Train on 2961 samples, validate on 329 samples
Epoch 1/30
2961/2961 - 162s - loss: 0.5509 - acc: 0.7376 - val_loss: 0.4377 - val_acc: 0.8328
Epoch 2/30
2961/2961 - 142s - loss: 0.4246 - acc: 0.8234 - val_loss: 0.4133 - val_acc: 0.8237
Epoch 3/30
2961/2961 - 115s - loss: 0.3836 - acc: 0.8322 - val_loss: 0.4121 - val_acc: 0.8237
Epoch 4/30
2961/2961 - 22s - loss: 0.3520 - acc: 0.8524 - val_loss: 0.4193 - val_acc: 0.8450
Epoch 5/30
2961/2961 - 132s - loss: 0.3282 - acc: 0.8625 - val_loss: 0.4175 

Epoch 21/30
2961/2961 - 143s - loss: 0.1181 - acc: 0.9595 - val_loss: 0.5963 - val_acc: 0.8146
Epoch 22/30
2961/2961 - 143s - loss: 0.1061 - acc: 0.9659 - val_loss: 0.5950 - val_acc: 0.8389
Epoch 23/30
2961/2961 - 104s - loss: 0.1092 - acc: 0.9659 - val_loss: 0.6168 - val_acc: 0.8267
Epoch 24/30
2961/2961 - 29s - loss: 0.0949 - acc: 0.9720 - val_loss: 0.6221 - val_acc: 0.8237
Epoch 25/30
2961/2961 - 141s - loss: 0.0966 - acc: 0.9710 - val_loss: 0.6181 - val_acc: 0.8237
Epoch 26/30
2961/2961 - 143s - loss: 0.0755 - acc: 0.9787 - val_loss: 0.6604 - val_acc: 0.8146
Epoch 27/30
2961/2961 - 142s - loss: 0.0726 - acc: 0.9780 - val_loss: 0.6982 - val_acc: 0.8207
Epoch 28/30
2961/2961 - 41s - loss: 0.0594 - acc: 0.9838 - val_loss: 0.7229 - val_acc: 0.8267
Epoch 29/30
2961/2961 - 93s - loss: 0.0601 - acc: 0.9811 - val_loss: 0.7180 - val_acc: 0.8237
Epoch 30/30
2961/2961 - 143s - loss: 0.0642 - acc: 0.9804 - val_loss: 0.6961 - val_acc: 0.8328
logs/DL/bidirectional/kfold7
Train on 2961 samples, v

Epoch 16/30
2961/2961 - 142s - loss: 0.1382 - acc: 0.9591 - val_loss: 0.5711 - val_acc: 0.8176
Epoch 17/30
2961/2961 - 36s - loss: 0.1366 - acc: 0.9578 - val_loss: 0.5625 - val_acc: 0.8055
Epoch 18/30
2961/2961 - 94s - loss: 0.1188 - acc: 0.9618 - val_loss: 0.5640 - val_acc: 0.8116
Epoch 19/30
2961/2961 - 142s - loss: 0.1130 - acc: 0.9689 - val_loss: 0.6647 - val_acc: 0.7903
Epoch 20/30
2961/2961 - 142s - loss: 0.1085 - acc: 0.9642 - val_loss: 0.6267 - val_acc: 0.8024
Epoch 21/30
2961/2961 - 21s - loss: 0.0980 - acc: 0.9730 - val_loss: 0.6073 - val_acc: 0.8085
Epoch 22/30
2961/2961 - 113s - loss: 0.0867 - acc: 0.9784 - val_loss: 0.7030 - val_acc: 0.7842
Epoch 23/30
2961/2961 - 143s - loss: 0.0922 - acc: 0.9726 - val_loss: 0.6961 - val_acc: 0.7872
Epoch 24/30
2961/2961 - 142s - loss: 0.0914 - acc: 0.9730 - val_loss: 0.6604 - val_acc: 0.7933
Epoch 25/30
2961/2961 - 43s - loss: 0.0836 - acc: 0.9780 - val_loss: 0.6857 - val_acc: 0.8024
Epoch 26/30
2961/2961 - 87s - loss: 0.0728 - acc: 0.97

In [157]:
process_results('bidirectional')

mean
 acc         0.928475
loss        0.189918
val_acc     0.822442
val_loss    0.516847
dtype: float64
std
 acc         0.056177
loss        0.119027
val_acc     0.021195
val_loss    0.117344
dtype: float64


media
 acc       0.816596
f1        0.843058
mse       0.183404
recall    0.839131
dtype: float64
std
 acc       0.012954
f1        0.007255
mse       0.012954
recall    0.028634
dtype: float64


In [72]:
bidi_pred = compute_full_model(convolutional1d_model, 'bidirectional', batch_size=8, epochs=30, shuffle=False, verbose=1)
compute_metrics(bidi_pred, y_test)

Train on 3290 samples
Epoch 1/30
3290/3290 [==============================] - 16s 5ms/sample - loss: 0.5223 - acc: 0.72615s - lo
Epoch 2/30
3290/3290 [==============================] - 14s 4ms/sample - loss: 0.3743 - acc: 0.8380
Epoch 3/30
3290/3290 [==============================] - 14s 4ms/sample - loss: 0.2985 - acc: 0.8763
Epoch 4/30
3290/3290 [==============================] - 14s 4ms/sample - loss: 0.2384 - acc: 0.9112
Epoch 5/30
3290/3290 [==============================] - 14s 4ms/sample - loss: 0.1857 - acc: 0.9389
Epoch 6/30
3290/3290 [==============================] - 14s 4ms/sample - loss: 0.1420 - acc: 0.9574
Epoch 7/30
3290/3290 [==============================] - 13s 4ms/sample - loss: 0.1132 - acc: 0.9666
Epoch 8/30
3290/3290 [==============================] - 14s 4ms/sample - loss: 0.0864 - acc: 0.9754
Epoch 9/30
3290/3290 [==============================] - 14s 4ms/sample - loss: 0.0702 - acc: 0.9833
Epoch 10/30
3290/3290 [==============================] - 14s 4ms/sample

{'mse': 0.18936170212765957,
 'recall': 0.8896746817538896,
 'f1': 0.8249180327868852,
 'acc': 0.8106382978723404}